In [1]:
import torch.nn as nn

# -------------------------------------------------------------------------
# Monkey-patch MultiheadAttention so forward() returns only `attn_output`
# -------------------------------------------------------------------------
_original_mha_forward = nn.MultiheadAttention.forward

def _mha_forward_no_weights(self, query, key, value, *args, **kwargs):
    # call the original, then discard the weights
    attn_output, _ = _original_mha_forward(self, query, key, value, *args, **kwargs)
    return attn_output

nn.MultiheadAttention.forward = _mha_forward_no_weights


In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
CTNet-Mamba: A Convolution-Mamba Network for EEG-Based Motor Imagery Classification

author: zhaowei701@163.com
"""

import os
import time
import random

import numpy as np
from zeta.nn import MultiQueryAttention

import torch
import torch.nn as nn
from torch.backends import cudnn
from torchsummary import summary

from einops import rearrange
from einops.layers.torch import Rearrange

from utils import load_data_evaluate, numberClassChannel, calMetrics

# Try to import real MambaBlock (fallback if missing)
try:
    from mamba_ssm import Mamba
except ImportError:
    class Mamba(nn.Module):
        def __init__(self, d_model, d_state=16, d_conv=4, expand=2):
            super().__init__()
            self.linear = nn.Linear(d_model, d_model)
        def forward(self, x):
            return self.linear(x)

from zeta.nn import MambaBlock, FeedForward, MultiQueryAttention

# -----------------------------------------------------------------------------
# PatchEmbeddingCNN
# -----------------------------------------------------------------------------
class PatchEmbeddingCNN(nn.Module):
    def __init__(self, f1=16, kernel_size=64, D=2,
                 pooling_size1=8, pooling_size2=8,
                 dropout_rate=0.3, number_channel=22, emb_size=40):
        super().__init__()
        f2 = D * f1
        self.cnn_module = nn.Sequential(
            nn.Conv2d(1, f1, (1, kernel_size), padding='same', bias=False),
            nn.BatchNorm2d(f1),
            nn.Conv2d(f1, f2, (number_channel, 1), groups=f1, bias=False),
            nn.BatchNorm2d(f2),
            nn.ELU(),
            nn.AvgPool2d((1, pooling_size1)),
            nn.Dropout(dropout_rate),
            nn.Conv2d(f2, f2, (1, 16), padding='same', bias=False),
            nn.BatchNorm2d(f2),
            nn.ELU(),
            nn.AvgPool2d((1, pooling_size2)),
            nn.Dropout(dropout_rate),
        )
        self.projection = Rearrange('b e h w -> b (h w) e')

    def forward(self, x):
        x = self.cnn_module(x)        # [B, f2, 1, seq]
        return self.projection(x)     # [B, seq, f2]

# -----------------------------------------------------------------------------
# LinearAttention
# -----------------------------------------------------------------------------
def exists(val): return val is not None

class LinearAttention(nn.Module):
    def __init__(self, dim, *, heads=4, dim_head=64, dropout=0.0):
        super().__init__()
        inner = heads * dim_head
        self.heads, self.scale = heads, dim_head**-0.5
        self.to_qkv = nn.Linear(dim, inner*3, bias=False)
        self.to_out = nn.Sequential(nn.Linear(inner, dim), nn.Dropout(dropout))

    def forward(self, x, mask=None):
        h = self.heads
        q,k,v = self.to_qkv(x).chunk(3, dim=-1)
        q,k,v = map(lambda t: rearrange(t, 'b n (h d) -> (b h) n d', h=h), (q,k,v))
        q, k = q*self.scale, k
        q, k = q.softmax(dim=-1), k.softmax(dim=-2)
        if exists(mask):
            mask = rearrange(mask, 'b n -> (b h) n', h=h)
            k = k.masked_fill(~mask.unsqueeze(-1), 0.)
        ctx = torch.einsum('b n d, b n e -> b d e', q, k)
        out = torch.einsum('b d e, b n d -> b n e', ctx, v)
        out = rearrange(out, '(b h) n d -> b n (h d)', h=h)
        return self.to_out(out)

# -----------------------------------------------------------------------------
# TransformerBlock
# -----------------------------------------------------------------------------
import torch
import torch.nn as nn

class MambaTransformerblock(nn.Module):
    def __init__(self, dim, heads, dim_head, dropout=0.1, 
                 ff_mult=4, d_state=16, depth=4):        
        super().__init__()
        self.layers = nn.ModuleList([
            self._create_layer(dim, heads, ff_mult, d_state, dropout)
            for _ in range(depth)
        ])
        self.norm = nn.LayerNorm(dim)

    def _create_layer(self, dim, heads, ff_mult, d_state, dropout):
        return nn.ModuleDict({
            # 1) Mamba
            'mamba_norm':   nn.LayerNorm(dim),
            'mamba':        MambaBlock(dim, d_state=d_state),
            'mamba_dropout':nn.Dropout(dropout),

            # 2) PyTorch MultiheadAttention
            'attn_norm':    nn.LayerNorm(dim),
            'attn':         nn.MultiheadAttention(embed_dim=dim,
                                                  num_heads=heads,
                                                  dropout=dropout,
                                                  batch_first=True),
            'attn_dropout': nn.Dropout(dropout),

            # 3) FFN
            'ffn_norm':     nn.LayerNorm(dim),
            'feedforward':  nn.Sequential(
                                 nn.Linear(dim, ff_mult * dim),
                                 nn.GELU(),
                                 nn.Dropout(dropout),
                                 nn.Linear(ff_mult * dim, dim)
                             ),
            'ffn_dropout':  nn.Dropout(dropout),
        })

    def forward(self, x):
        # x: [B, L, dim]
        for layer in self.layers:
            # (1) Mamba sublayer
            m = layer['mamba_norm'](x)
            x = x + layer['mamba_dropout'](layer['mamba'](m))

            # (2) Multi-head self-attention sublayer
            a = layer['attn_norm'](x)
            # because of the monkey-patch, this returns a Tensor directly
            attn_out = layer['attn'](a, a, a, need_weights=False)
            x = x + layer['attn_dropout'](attn_out)

            # (3) Feed-forward sublayer
            f = layer['ffn_norm'](x)
            x = x + layer['ffn_dropout'](layer['feedforward'](f))

        return self.norm(x)




# -----------------------------------------------------------------------------
# EEGMambaTransformer
# -----------------------------------------------------------------------------
class EEGMambaTransformer(nn.Module):
    def __init__(self, emb_size=40, depth=6, heads=4,
                 d_state=16, transformer_depth=1, mamba_depth=3,
                 database_type='A', eeg1_f1=8, eeg1_kernel_size=64,
                 eeg1_D=2, eeg1_pooling_size1=8, eeg1_pooling_size2=8,
                 eeg1_dropout_rate=0.5, flatten_eeg1=15*16):
        super().__init__()
        self.number_class, self.number_channel = numberClassChannel(database_type)

        self.cnn = PatchEmbeddingCNN(
            f1=eeg1_f1, kernel_size=eeg1_kernel_size,
            D=eeg1_D, pooling_size1=eeg1_pooling_size1,
            pooling_size2=eeg1_pooling_size2,
            dropout_rate=eeg1_dropout_rate,
            number_channel=self.number_channel,
            emb_size=emb_size
        )

        dim_head = emb_size // heads

        self.mamba_transformer = MambaTransformerblock(
            dim=emb_size, heads=heads,
            dim_head=dim_head,
            dropout=eeg1_dropout_rate, ff_mult=4,
            d_state=d_state,
            depth = depth,
        )

        self.flatten = nn.Flatten()
        self.classification = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(flatten_eeg1, self.number_class)
        )

    def forward(self, x):
        x = self.cnn(x)                       # [B, seq, emb]
        x = x * np.sqrt(x.shape[-1])
        x = self.mamba_transformer(x)
        return x, self.classification(self.flatten(x))

# -----------------------------------------------------------------------------
# Experiment wrapper
# -----------------------------------------------------------------------------
class ExP:
    def __init__(self, nsub, data_dir, result_name,
                 epochs=300, number_aug=3, number_seg=8,
                 evaluate_mode='subject-dependent',
                 heads=2, emb_size=16, depth=6,
                 d_state=16, transformer_depth=1, mamba_depth=3,
                 dataset_type='A', eeg1_f1=8, eeg1_kernel_size=64,
                 eeg1_D=2, eeg1_pooling_size1=8, eeg1_pooling_size2=8,
                 eeg1_dropout_rate=0.5, flatten_eeg1=15*16,
                 validate_ratio=0.3, learning_rate=1e-3, batch_size=72,
                 early_stopping_patience=100 ):
        self.nSub = nsub
        self.dataset_type = dataset_type
        self.data_dir = data_dir
        self.result_name = result_name
        self.n_epochs = epochs
        self.patience = early_stopping_patience
        self.no_improve = 0
        self.best_loss = float('inf')
        self.batch_size = batch_size
        self.validate_ratio = validate_ratio

        self.criterion = nn.CrossEntropyLoss().cuda()

        self.model = EEGMambaTransformer(
            emb_size=emb_size, depth=depth, heads=heads,
            d_state=d_state,
            transformer_depth=transformer_depth,
            mamba_depth=mamba_depth,
            database_type=dataset_type,
            eeg1_f1=eeg1_f1, eeg1_kernel_size=eeg1_kernel_size,
            eeg1_D=eeg1_D, eeg1_pooling_size1=eeg1_pooling_size1,
            eeg1_pooling_size2=eeg1_pooling_size2,
            eeg1_dropout_rate=eeg1_dropout_rate,
            flatten_eeg1=flatten_eeg1
        ).cuda()

        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=learning_rate)
        os.makedirs(self.result_name, exist_ok=True)
        self.model_filename = os.path.join(self.result_name, f"model_{self.nSub}.pth")

    def interaug(self, timg, label):
        aug_data, aug_label = [], []
        recs = 3 * (self.batch_size // self.model.number_class)
        segpts = 1000 // 8
        for cls in range(self.model.number_class):
            idx = np.where(label == cls + 1)
            data, lbl = timg[idx], label[idx]
            tmp = np.zeros((recs,1,self.model.number_channel,1000), dtype=np.float32)
            for i in range(recs):
                for j in range(8):
                    ridx = random.randrange(data.shape[0])
                    tmp[i,0,:,j*segpts:(j+1)*segpts] = data[ridx,0,:,j*segpts:(j+1)*segpts]
            aug_data.append(tmp); aug_label.append(lbl[:recs])
        aug_data = np.concatenate(aug_data).astype(np.float32)
        aug_label = np.concatenate(aug_label)
        perm = np.random.permutation(len(aug_data))
        return (
            torch.from_numpy(aug_data[perm]).cuda(),
            torch.from_numpy((aug_label[perm]-1)).long().cuda()
        )

    def get_source_data(self):
        tr_x, tr_y, te_x, te_y = load_data_evaluate(
            self.data_dir, self.dataset_type, self.nSub,
            mode_evaluate='subject-dependent'
        )
        tr_x = np.expand_dims(tr_x, axis=1).astype(np.float32)
        te_x = np.expand_dims(te_x, axis=1).astype(np.float32)
        tr_y = tr_y.reshape(-1)
        te_y = te_y.reshape(-1)
        m, s = tr_x.mean(), tr_x.std()
        tr_x = (tr_x - m) / s
        te_x = (te_x - m) / s
        return tr_x, tr_y, te_x, te_y

    def train(self):
        tr_x, tr_y, te_x, te_y = self.get_source_data()
    
        # Create validation split
        dataset_size = len(tr_x)
        val_size = int(self.validate_ratio * dataset_size)
        train_size = dataset_size - val_size
        train_ds, val_ds = torch.utils.data.random_split(
            torch.utils.data.TensorDataset(torch.from_numpy(tr_x), torch.from_numpy(tr_y-1)),
            [train_size, val_size]
        )
    
        test_ds = torch.utils.data.TensorDataset(torch.from_numpy(te_x), torch.from_numpy(te_y-1))
        self.test_loader = torch.utils.data.DataLoader(test_ds, batch_size=self.batch_size)
    
        best_loss = float('inf')
        for e in range(self.n_epochs):
            # Training
            self.model.train()
            train_loader = torch.utils.data.DataLoader(train_ds, batch_size=self.batch_size, shuffle=True)
            
            # Initialize training metrics
            epoch_train_loss = 0.0
            train_correct = 0
            total_samples = 0
            
            # Start timing and memory tracking
            torch.cuda.synchronize()
            start_time = time.time()
            torch.cuda.reset_peak_memory_stats()
            
            for xb, yb in train_loader:
                xb, yb = xb.cuda().float(), yb.cuda().long()
                aug_x, aug_y = self.interaug(tr_x, tr_y)
                xb = torch.cat([xb, aug_x])
                yb = torch.cat([yb, aug_y])
                
                _, out = self.model(xb)
                loss = self.criterion(out, yb)
                
                # Backprop
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                
                # Track metrics
                epoch_train_loss += loss.item()
                preds = out.argmax(dim=1)
                train_correct += (preds == yb).sum().item()
                total_samples += yb.size(0)
    
            # Calculate training metrics
            train_loss = epoch_train_loss / len(train_loader)
            train_acc = train_correct / total_samples
            
            # Calculate memory and speed
            torch.cuda.synchronize()
            elapsed = time.time() - start_time
            mem_used = torch.cuda.max_memory_allocated() / (1024 ** 2)  # MB
            speed = total_samples / elapsed if elapsed > 0 else 0
    
            # Validation
            self.model.eval()
            val_loss = 0.0
            val_correct = 0
            with torch.no_grad():
                val_loader = torch.utils.data.DataLoader(val_ds, batch_size=self.batch_size)
                for xb, yb in val_loader:
                    xb, yb = xb.cuda().float(), yb.cuda().long()
                    _, out = self.model(xb)
                    loss = self.criterion(out, yb)
                    val_loss += loss.item()
                    val_correct += (out.argmax(1) == yb).sum().item()
    
            val_loss = val_loss / len(val_loader)
            val_acc = val_correct / len(val_ds)
    
            print(f"Epoch {e+1}/{self.n_epochs} | "
                  f"Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | "
                  f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f} | "
                  f"Mem: {mem_used:.2f}MB | Speed: {speed:.2f} samples/s")
    
            if val_loss < best_loss:
                best_loss = val_loss
                torch.save(self.model.state_dict(), self.model_filename)
            else:
                self.no_improve += 1
                if self.no_improve >= self.patience:
                    print(f"Stopping early at epoch {e+1} (no improvement in {self.patience} epochs).")
                    break

    # Rest of test evaluation...

    # Rest of test evaluation remains the same...

        self.model.load_state_dict(torch.load(self.model_filename))
        self.model.eval()
        preds, trues = [], []
        with torch.no_grad():
            for xb, yb in self.test_loader:
                xb = xb.cuda()
                _, out = self.model(xb)
                preds.append(out.argmax(1).cpu().numpy())
                trues.append(yb.numpy())
        preds = np.concatenate(preds); trues = np.concatenate(trues)
        acc, *_ = calMetrics(trues, preds)
        print(f"Subject {self.nSub} final accuracy: {acc:.4f}")
        return acc

# -----------------------------------------------------------------------------
def main(result_dir, DATA_DIR, N_SUBJECT, **cfg):
    os.makedirs(result_dir, exist_ok=True)

    number_class, number_channel = numberClassChannel(cfg['dataset_type'])
    model = EEGMambaTransformer(
        emb_size=cfg['emb_size'], depth=cfg['depth'], heads=cfg['heads'],
        d_state=cfg['d_state'],
        transformer_depth=cfg['transformer_depth'],
        mamba_depth=cfg['mamba_depth'],
        database_type=cfg['dataset_type'],
        eeg1_f1=cfg['eeg1_f1'], eeg1_kernel_size=cfg['eeg1_kernel_size'],
        eeg1_D=cfg['eeg1_D'], eeg1_pooling_size1=cfg['eeg1_pooling_size1'],
        eeg1_pooling_size2=cfg['eeg1_pooling_size2'],
        eeg1_dropout_rate=cfg['eeg1_dropout_rate'],
        flatten_eeg1=cfg['flatten_eeg1']
    ).cuda()
    summary(model, (1, number_channel, 1000))
    print(time.asctime(time.localtime(time.time())))

    accs = []
    for sub in range(1, N_SUBJECT+1):
        seed_n = np.random.randint(2024)
        random.seed(seed_n); np.random.seed(seed_n)
        torch.manual_seed(seed_n); torch.cuda.manual_seed(seed_n)
        torch.cuda.manual_seed_all(seed_n)

        print(f"seed is {seed_n}")
        print(f"Subject {sub}")

        exp = ExP(sub, DATA_DIR, result_dir, **cfg)
        accs.append(exp.train())

    print("Average accuracy:", np.mean(accs))
    return accs

if __name__ == "__main__":
    cudnn.benchmark = False
    cudnn.deterministic = True

    CONFIG = dict(
        emb_size=256, depth=1, heads=4,
        d_state=64, transformer_depth=1, mamba_depth=2,
        dataset_type='A',
        eeg1_f1=8, eeg1_kernel_size=64,
        eeg1_D=32, eeg1_pooling_size1=8, eeg1_pooling_size2=8,
        eeg1_dropout_rate=0.6, flatten_eeg1=15*256,
        epochs=1000, number_aug=3, number_seg=8,
        validate_ratio=0.3, learning_rate=1e-3, batch_size=36
    )

    DATA_DIR   = "bci2a/"
    RESULT_DIR = f"CTNet_Mamba_{int(time.time())}"
    N_SUBJECT  = 9

    main(RESULT_DIR, DATA_DIR, N_SUBJECT, **CONFIG)
    print("Done.")


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 8, 22, 1000]             512
       BatchNorm2d-2          [-1, 8, 22, 1000]              16
            Conv2d-3         [-1, 256, 1, 1000]           5,632
       BatchNorm2d-4         [-1, 256, 1, 1000]             512
               ELU-5         [-1, 256, 1, 1000]               0
         AvgPool2d-6          [-1, 256, 1, 125]               0
           Dropout-7          [-1, 256, 1, 125]               0
            Conv2d-8          [-1, 256, 1, 125]       1,048,576
       BatchNorm2d-9          [-1, 256, 1, 125]             512
              ELU-10          [-1, 256, 1, 125]               0
        AvgPool2d-11           [-1, 256, 1, 15]               0
          Dropout-12           [-1, 256, 1, 15]               0
        Rearrange-13              [-1, 15, 256]               0
PatchEmbeddingCNN-14              [-1, 

In [3]:
# Import necessary libraries
import torch
import torch.nn as nn
from einops.layers.torch import Rearrange
import torch.nn.functional as F

# Define the CNN model
class PatchEmbeddingCNN(nn.Module):
    def __init__(self, f1=16, kernel_size=64, D=2, pooling_size1=8, pooling_size2=8, dropout_rate=0.3, number_channel=22, emb_size=40):
        super().__init__()
        f2 = D * f1
        self.cnn_module = nn.Sequential(
            # Temporal convolution (kernel size = 64)
            nn.Conv2d(1, f1, (1, kernel_size), (1, 1), padding='same', bias=False), # [batch, 1, 22, 1000]
            nn.BatchNorm2d(f1),
            # Channel depth-wise convolution
            nn.Conv2d(f1, f2, (number_channel, 1), (1, 1), groups=f1, padding='valid', bias=False), 
            nn.BatchNorm2d(f2),
            nn.ELU(),
            # Average pooling to obtain 'patches' along the time dimension (as in ViT)
            nn.AvgPool2d((1, pooling_size1)),
            nn.Dropout(dropout_rate),
            # Additional spatial convolution
            nn.Conv2d(f2, f2, (1, 16), padding='same', bias=False), 
            nn.BatchNorm2d(f2),
            nn.ELU(),
            # Final pooling
            nn.AvgPool2d((1, pooling_size2)),
            nn.Dropout(dropout_rate),
        )

        # Projection to reshape the tensor to match the desired output format
        self.projection = nn.Sequential(
            Rearrange('b e h w -> b (h w) e'),
        )
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        b, _, _, _ = x.shape
        x = self.cnn_module(x)  # Pass through CNN layers
        x = self.projection(x)   # Flatten and rearrange to (batch, seq_len, emb_size)
        return x

# Create a dummy input tensor (example: 288 samples, 1 channel, 22 features, 1000 time steps)
input_data = torch.randn(1, 1, 22, 1000)  # Random example data

# Initialize the CNN model
cnn_model = PatchEmbeddingCNN(f1=8, kernel_size=64, D=2, pooling_size1=8, pooling_size2=8, dropout_rate=0.3, number_channel=22, emb_size=40)

# Pass the input data through the CNN model
output_data = cnn_model(input_data)

# Print the shape of the output data
print(f"Input data shape: {input_data.shape}")
print(f"Output data shape after CNN: {output_data.shape}")
print(output_data)


Input data shape: torch.Size([1, 1, 22, 1000])
Output data shape after CNN: torch.Size([1, 15, 16])
tensor([[[ 4.4427e-02,  1.6865e-01,  0.0000e+00,  4.4497e-01,  1.6248e-01,
           2.2551e-02,  2.1011e-01,  1.3673e-01,  7.4656e-01,  5.7626e-01,
           0.0000e+00, -0.0000e+00,  2.6901e-01,  6.1869e-01,  5.1004e-01,
           1.1605e-01],
         [ 3.4188e-01, -2.1637e-01,  9.2026e-02,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  9.8942e-01, -0.0000e+00, -6.7854e-01,  6.5418e-01,
           7.0507e-01,  3.0271e-01,  1.4244e-02, -0.0000e+00,  1.9596e-01,
          -0.0000e+00],
         [-2.0928e-01,  4.5755e-01,  3.6308e-01,  6.7305e-01,  3.4740e-01,
          -3.2171e-01, -4.9382e-01,  2.3126e-01, -2.5969e-01,  9.1415e-02,
           1.4810e-01,  6.9497e-01,  6.7038e-01,  2.3976e-01, -6.3225e-01,
           0.0000e+00],
         [ 7.4252e-01,  1.6591e-02,  0.0000e+00,  4.3490e-02, -4.5228e-02,
           1.0913e-01, -4.7366e-01,  6.4331e-01,  0.0000e+00, -2.3919e-01,
   

In [4]:
import torch
torch.cuda.max_memory_allocated()

2890110464

In [5]:
import torch
from mamba_transformer import MambaTransformer
# Create an instance of the MambaTransformer model
model = MambaTransformer(
    num_tokens=100,  # Number of tokens in the input sequence
    dim=512,  # Dimension of the model
    heads=8,  # Number of attention heads
    depth=4,  # Number of transformer layers
    dim_head=64,  # Dimension of each attention head
    d_state=512,  # Dimension of the state
    dropout=0.1,  # Dropout rate
    ff_mult=4,  # Multiplier for the feed-forward layer dimension
    return_embeddings=False,  # Whether to return the embeddings,
    transformer_depth=2,  # Number of transformer blocks
    mamba_depth=10,  # Number of Mamba blocks,
    use_linear_attn=True,  # Whether to use linear attention
)

# Pass the input tensor through the model and print the output shape
output_data = output_data.long()  # Convert the tensor to Long type

out = model(output_data)

print(out.shape)

torch.Size([1, 15, 16, 100])
